# Description

The goal of this notebook is very simple: it just lists the content (traits/diseases) that belong to each cluster across all selected "best partitions". Although one would take a look at them here to check whether clusters of traits make sense, that analysis is carried out first by looking at the clustering trees (which are generated later). Then, this notebooks serves as a simple list with the content of the clusters.

# Modules loading

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from IPython.display import display
from pathlib import Path

import pandas as pd

from utils import generate_result_set_name
import conf

# Settings

In [3]:
CONSENSUS_CLUSTERING_DIR = Path(
    conf.RESULTS["CLUSTERING_DIR"], "consensus_clustering"
).resolve()

display(CONSENSUS_CLUSTERING_DIR)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/clustering/consensus_clustering')

## Load data

In [4]:
INPUT_SUBSET = "umap"

In [5]:
INPUT_STEM = "z_score_std-projection-smultixcan-efo_partial-mashr-zscores"

In [6]:
DR_OPTIONS = {
    "n_components": 5,
    "metric": "euclidean",
    "n_neighbors": 15,
    "random_state": 0,
}

In [7]:
input_filepath = Path(
    conf.RESULTS["DATA_TRANSFORMATIONS_DIR"],
    INPUT_SUBSET,
    generate_result_set_name(
        DR_OPTIONS, prefix=f"{INPUT_SUBSET}-{INPUT_STEM}-", suffix=".pkl"
    ),
).resolve()
display(input_filepath)

assert input_filepath.exists(), "Input file does not exist"

input_filepath_stem = input_filepath.stem
display(input_filepath_stem)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/data_transformations/umap/umap-z_score_std-projection-smultixcan-efo_partial-mashr-zscores-metric_euclidean-n_components_5-n_neighbors_15-random_state_0.pkl')

'umap-z_score_std-projection-smultixcan-efo_partial-mashr-zscores-metric_euclidean-n_components_5-n_neighbors_15-random_state_0'

In [8]:
data_umap = pd.read_pickle(input_filepath)

In [9]:
data_umap.shape

(3752, 5)

In [10]:
data_umap.head()

,UMAP1,UMAP2,UMAP3,UMAP4,UMAP5
100001_raw-Food_weight,0.426554,0.670532,7.363805,1.171837,6.297295
100002_raw-Energy,-1.605179,0.815699,8.288521,0.990394,6.817351
100003_raw-Protein,-1.656178,0.788297,8.355906,1.017072,6.845651
100004_raw-Fat,-1.508325,0.802536,8.328274,1.033939,6.709319
100005_raw-Carbohydrate,-1.617872,0.812711,8.307973,1.020575,6.825944


# Load best partitions

In [11]:
input_file = Path(CONSENSUS_CLUSTERING_DIR, "best_partitions_by_k.pkl").resolve()
display(input_file)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/clustering/consensus_clustering/best_partitions_by_k.pkl')

In [12]:
best_partitions = pd.read_pickle(input_file)

In [13]:
best_partitions.shape

(59, 4)

In [14]:
best_partitions.head()

,method,partition,ari_median,selected
k,,,,
14,scc_025,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.090117,True
22,scc_020,"[13, 18, 18, 18, 18, 18, 18, 18, 18, 13, 18, 1...",0.090100,True
13,scc_025,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.089920,True
12,scc_025,"[9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, ...",0.089870,True
11,scc_025,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.089616,True


# Analysis of clusterings

In [15]:
from IPython.display import HTML

In [16]:
def show_cluster_stats(clustering_data, selected_partition, selected_cluster):
    traits = [t for t in clustering_data[selected_partition == selected_cluster].index]
    display(len(traits))
    display(traits)

In [17]:
selected_k_values = best_partitions[best_partitions["selected"]].index.tolist()
selected_k_values.sort()
display(selected_k_values)

[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 22, 25, 26]

In [18]:
for k in selected_k_values:
    display(HTML(f"<h2>Partition with k={k}</h2>"))
    display(best_partitions.loc[k])

    part = best_partitions.loc[k, "partition"]
    display(part.shape)

    part_stats = pd.Series(part).value_counts()
    display(part_stats)

    for cluster_number in part_stats.index.sort_values():
        display(HTML(f"<h3>Cluster {k}.{cluster_number}</h3>"))

        cluster_traits = data_umap[part == cluster_number].index
        display(len(cluster_traits))
        display(cluster_traits)

method                                                  scc_030
partition     [0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 4, 4, 0, 0, 0, ...
ari_median                                             0.087086
selected                                                   True
Name: 5, dtype: object

(3752,)

0    3233
4     445
3      47
2      15
1      12
dtype: int64

3233

Index(['100001_raw-Food_weight', '100002_raw-Energy', '100003_raw-Protein',
       '100004_raw-Fat', '100005_raw-Carbohydrate', '100006_raw-Saturated_fat',
       '100007_raw-Polyunsaturated_fat', '100008_raw-Total_sugars',
       '100010-Portion_size', '100013_raw-Vitamin_B12',
       ...
       'viral disease', 'viral human hepatitis infection', 'viral meningitis',
       'visual impairment', 'vitiligo', 'vitreous body disease',
       'vocal cord polyp', 'voice disorders', 'whooping cough',
       'wrist fracture'],
      dtype='object', length=3233)

...

Index(['rheumatoid arthritis', 'systemic lupus erythematosus'], dtype='object')

101

Index(['41248_1000-Destinations_on_discharge_from_hospital_recoded_Usual_Place_of_residence',
       '4194_raw-Pulse_rate', '4260-Round_of_numeric_memory_test',
       '4282-Maximum_digits_remembered_correctly',
       '4283-Number_of_rounds_of_numeric_memory_test_performed',
       '4290_raw-Duration_screen_displayed', '4291-Number_of_attempts',
       '46_raw-Hand_grip_strength_left', '47_raw-Hand_grip_strength_right',
       '4825-Noisy_workplace',
       ...
       '864-Number_of_daysweek_walked_10_minutes', '874_raw-Duration_of_walks',
       '884-Number_of_daysweek_of_moderate_physical_activity_10_minutes',
       '894-Duration_of_moderate_activity',
       '904-Number_of_daysweek_of_vigorous_physical_activity_10_minutes',
       '924-Usual_walking_pace',
       '943-Frequency_of_stair_climbing_in_last_4_weeks',
       '971-Frequency_of_walking_for_pleasure_in_last_4_weeks',
       'ICDMAIN_ANY_ENTRY-Any_ICDMAIN_event_in_hilmo_or_causes_of_death',
       'episodic memory'],
     

147

Index(['1021-Duration_of_light_DIY', '102_raw-Pulse_rate_automated_reading',
       '1031-Frequency_of_friendfamily_visits',
       '1050-Time_spend_outdoors_in_summer',
       '1060-Time_spent_outdoors_in_winter',
       '1070-Time_spent_watching_television_TV',
       '1080-Time_spent_using_computer', '1090-Time_spent_driving',
       '1100-Drive_faster_than_motorway_speed_limit',
       '110001-Invitation_to_complete_online_24hour_recall_dietary_questionnaire_acceptance',
       ...
       'self rated health', 'self reported educational attainment',
       'sleep duration', 'smoking behavior', 'smoking cessation',
       'snoring measurement', 'tiredness measurement', 'varicose veins',
       'wheezing', 'worry measurement'],
      dtype='object', length=147)